In [1]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''  # noqa

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/luokui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [2]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')

Build model...


In [7]:
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

#  如果 return_sequences=True 值为 7 * 128, 否则就是最后一个 time的 输出 None * 128

print(model.summary())

# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(2): #(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
# model.add(layers.Dense(len(chars)))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               72192     
Total params: 72,192
Trainable params: 72,192
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
lstm_13 (LSTM)               (None, 4, 128)            131584    
____________________________________________________

In [8]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 245us/step - loss: 1.9190 - acc: 0.3150 - val_loss: 1.8079 - val_acc: 0.3356
Q 513+855 T 1368 ☒ 144 
Q 989+32  T 1021 ☒ 105 
Q 70+186  T 256  ☒ 145 
Q 67+519  T 586  ☒ 145 
Q 8+621   T 629  ☒ 143 
Q 697+339 T 1036 ☒ 1447
Q 90+981  T 1071 ☒ 105 
Q 20+939  T 959  ☒ 145 
Q 44+395  T 439  ☒ 145 
Q 77+15   T 92   ☒ 14  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - loss: 1.7626 - acc: 0.3437 - val_loss: 1.7025 - val_acc: 0.3629
Q 38+107  T 145  ☒ 112 
Q 0+10    T 10   ☒ 11  
Q 532+611 T 1143 ☒ 1029
Q 87+9    T 96   ☒ 119 
Q 885+98  T 983  ☒ 101 
Q 215+3   T 218  ☒ 112 
Q 872+0   T 872  ☒ 889 
Q 375+3   T 378  ☒ 119 
Q 978+3   T 981  ☒ 809 
Q 1+971   T 972  ☒ 119 

------------------

45000/45000 [==============================] - 10s 212us/step - loss: 0.6745 - acc: 0.7454 - val_loss: 0.6663 - val_acc: 0.7471
Q 6+961   T 967  ☒ 960 
Q 177+362 T 539  ☒ 530 
Q 604+2   T 606  ☒ 607 
Q 737+98  T 835  ☒ 830 
Q 357+186 T 543  ☒ 544 
Q 480+8   T 488  ☒ 483 
Q 243+279 T 522  ☒ 524 
Q 95+402  T 497  ☒ 404 
Q 806+79  T 885  ☒ 883 
Q 202+9   T 211  ☒ 210 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 216us/step - loss: 0.6649 - acc: 0.7489 - val_loss: 0.6717 - val_acc: 0.7443
Q 6+628   T 634  ☒ 633 
Q 638+128 T 766  ☒ 763 
Q 847+96  T 943  ☒ 941 
Q 12+386  T 398  ☒ 394 
Q 922+610 T 1532 ☒ 1539
Q 99+25   T 124  ☒ 129 
Q 386+53  T 439  ☒ 431 
Q 14+45   T 59   ☒ 55  
Q 42+218  T 260  ☒ 259 
Q 797+944 T 1741 ☑ 1741

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 10s 223us/step - loss: 0.0024 - acc: 0.9999 - val_loss: 0.0034 - val_acc: 0.9993
Q 842+0   T 842  ☑ 842 
Q 204+283 T 487  ☑ 487 
Q 612+19  T 631  ☑ 631 
Q 903+61  T 964  ☑ 964 
Q 19+28   T 47   ☑ 47  
Q 740+83  T 823  ☑ 823 
Q 511+86  T 597  ☑ 597 
Q 62+220  T 282  ☑ 282 
Q 96+976  T 1072 ☑ 1072
Q 86+718  T 804  ☑ 804 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 223us/step - loss: 0.0025 - acc: 0.9997 - val_loss: 0.0059 - val_acc: 0.9984
Q 474+4   T 478  ☑ 478 
Q 47+991  T 1038 ☑ 1038
Q 118+681 T 799  ☒ 899 
Q 94+246  T 340  ☑ 340 
Q 652+233 T 885  ☑ 885 
Q 448+42  T 490  ☑ 490 
Q 274+48  T 322  ☑ 322 
Q 53+695  T 748  ☑ 748 
Q 8+463   T 471  ☑ 471 
Q 2+37    T 39   ☑ 39  

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

KeyboardInterrupt: 

In [3]:
# 列表追加的方式
a = [[1, 2], [2, 3]]
b = [[3, 4], [4, 5]]
a + b

[[1, 2], [2, 3], [3, 4], [4, 5]]

In [6]:
import keras.backend as k


In [31]:
a = np.array([[1, 1], [1, 2]])
b = np.array([[1, 0], [0, 1]])
c = np.array([[1, 1, 1, 0], [1, 2, 0, 1]])
zx = np.array([[10, 10]])

In [37]:
t1 = np.matmul(zx, a)
t1 

array([[20, 30]])

In [33]:
a

array([[1, 1],
       [1, 2]])

In [38]:
t2 = np.matmul(zx, b)
t2

array([[10, 10]])

In [35]:
b

array([[1, 0],
       [0, 1]])

In [36]:
np.matmul(zx, c)

array([[20, 30, 10, 10]])

In [39]:
t1 + t2

array([[30, 40]])

In [40]:
a

array([[1, 1],
       [1, 2]])

In [41]:
(a)

array([[1, 1],
       [1, 2]])

In [42]:
a

array([[1, 1],
       [1, 2]])

In [43]:
b

array([[1, 0],
       [0, 1]])

In [44]:
a * b # 逐步乘机。

array([[1, 0],
       [0, 2]])

In [45]:
a

array([[1, 1],
       [1, 2]])

In [51]:
[list(a)]

[[array([1, 1]), array([1, 2])]]

In [55]:
list(((1, 2),(2, 3)))

[(1, 2), (2, 3)]

In [56]:
a = [[1, 2], [3, 4]]
b = [[2, 3], [2, 2]]


In [57]:
a + b

[[1, 2], [3, 4], [2, 3], [2, 2]]